In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from fastai.tabular.core import df_shrink

train = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
test = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
#dw_train = pd.read_csv('/kaggle/input/dwtrain/newtrain.csv')

train = df_shrink(train)
#dw_train = df_shrink(dw_train)
test = df_shrink(test)

In [ ]:
# Concatenate train and test data
all_df = pd.concat([train, test], axis=0)
# Create new feature
all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
display(all_df)
# Split back up
train = all_df.iloc[:len(train)]
test = all_df.iloc[len(train):]
del all_df

There is no null values in test dataset

In [ ]:
from datetime import datetime

def add_date_feats(data):
    data["month"]=pd.to_datetime(data["startdate"]).dt.month
    data["week"]=pd.to_datetime(data["startdate"]).dt.week
    data["year"]=pd.to_datetime(data["startdate"]).dt.year
    data["day"]=pd.to_datetime(data["startdate"]).dt.day
    data["day_of_weeeks"]=pd.to_datetime(data["startdate"]).dt.day_of_week
    data["quarter"]=pd.to_datetime(data["startdate"]).dt.quarter
    data["hour"]=pd.to_datetime(data["startdate"]).dt.hour
    #data = data.drop(['startdate',"climateregions__climateregion"],axis=1)
    return data

dw_train = add_date_feats(dw_train)
train = add_date_feats(train)
test = add_date_feats(test)

Handling null values is not recommended for xgboost

In [ ]:
train['nmme0-prate-34w__ccsm30'] = train['nmme0-prate-34w__ccsm30'].fillna(train['nmme0-prate-34w__ccsm30'].mean())
train['nmme0-tmp2m-34w__ccsm30'] = train['nmme0-tmp2m-34w__ccsm30'].fillna(train['nmme0-tmp2m-34w__ccsm30'].mean())
train['ccsm30'] = train['ccsm30'].fillna(train['ccsm30'].mean())
train['nmme0-prate-56w__ccsm30'] = train['nmme0-prate-56w__ccsm30'].fillna(train['nmme0-prate-56w__ccsm30'].mean())
train['nmme-tmp2m-56w__ccsm3'] = train['nmme-tmp2m-56w__ccsm3'].fillna(train['nmme-tmp2m-56w__ccsm3'].mean())
train['nmme-prate-56w__ccsm3'] = train['nmme-prate-56w__ccsm3'].fillna(train['nmme-prate-56w__ccsm3'].mean())
train['nmme-tmp2m-34w__ccsm3'] = train['nmme-tmp2m-34w__ccsm3'].fillna(train['nmme-tmp2m-34w__ccsm3'].mean())
train['nmme-prate-34w__ccsm3'] = train['nmme-prate-34w__ccsm3'].fillna(train['nmme-prate-34w__ccsm3'].mean())

In [ ]:
#converting object type to category
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['climateregions__climateregion'] = le.fit_transform(train['climateregions__climateregion'])
test['climateregions__climateregion'] = le.transform(test['climateregions__climateregion'])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.svm import SVR

scaler = StandardScaler()
target="contest-tmp2m-14d__tmp2m"
exclude_cols = ['index', 'startdate', 'lat', 'lon', 'loc_group', 'climateregions__climateregion']
features = [c for c in train.columns if ((c != target) & (c not in exclude_cols))]
submission = pd.read_csv('/kaggle/input/widsdatathon2023/sample_solution.csv')
submission[target] = 0

#model definition
for loc_group in train['loc_group'].unique().tolist():
    scaler = StandardScaler()
    filtered_train = train[train['loc_group'] == loc_group]    
       
    X_train = scaler.fit_transform(filtered_train[features])
        
    y_train = filtered_train[target]

    SVR = SVR(kernel='rbf', C=1.0,cache_size=1000, verbose=True)

    SVR.fit(X_train, y_train)
    
    filtered_test = test[test['loc_group'] == loc_group]
    predicted = SVR.predict(scaler.transform(filtered_test[features]))
    
    submission[target].iloc[filtered_test.index] = predicted
    
submission.to_csv('/kaggle/working/submission.csv', index = False) # Set index to false to avoid issues in evaluation
